In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm 
from pyspark import SparkConf
from subprocess import check_output



matplotlib.rcParams['figure.figsize'] = (10,10)

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
sc.master

u'local[*]'

In [5]:
# Import regex module
import re, string
from operator import add

In [6]:
gb = sc.textFile("hdfs://localhost:54310/Youtube/GBvideos.csv")
de = sc.textFile("hdfs://localhost:54310/Youtube/DEvideos.csv")
us = sc.textFile("hdfs://localhost:54310/Youtube/USvideos.csv")
ca = sc.textFile("hdfs://localhost:54310/Youtube/CAvideos.csv")
fr = sc.textFile("hdfs://localhost:54310/Youtube/FRvideos.csv")

In [7]:
us.count()

48698

In [8]:
us.take(10)

[u'video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description',
 u'2kyS6SvSYSE,17.14.11,"WE WANT TO TALK ABOUT OUR MARRIAGE","CaseyNeistat",22,2017-11-13T17:13:01.000Z,"SHANtell martin",748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,"SHANTELL\'S CHANNEL - https://www.youtube.com/shantellmartin\\nCANDICE - https://www.lovebilly.com\\n\\nfilmed this video in 4k on this -- http://amzn.to/2sTDnRZ\\nwith this lens -- http://amzn.to/2rUJOmD\\nbig drone - http://tinyurl.com/h4ft3oy\\nOTHER GEAR ---  http://amzn.to/2o3GLX5\\nSony CAMERA http://amzn.to/2nOBmnv\\nOLD CAMERA; http://amzn.to/2o2cQBT\\nMAIN LENS; http://amzn.to/2od5gBJ\\nBIG SONY CAMERA; http://amzn.to/2nrdJRO\\nBIG Canon CAMERA; http://tinyurl.com/jn4q4vz\\nBENDY TRIPOD THING; http://tinyurl.com/gw3ylz2\\nYOU NEED THIS FOR THE BENDY TRIPOD; http://tinyurl.co

In [9]:
header = us.first()
print header

video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description


In [10]:
noheader = us.filter(lambda x: x<>header)

In [11]:
noheader.first()

u'2kyS6SvSYSE,17.14.11,"WE WANT TO TALK ABOUT OUR MARRIAGE","CaseyNeistat",22,2017-11-13T17:13:01.000Z,"SHANtell martin",748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,"SHANTELL\'S CHANNEL - https://www.youtube.com/shantellmartin\\nCANDICE - https://www.lovebilly.com\\n\\nfilmed this video in 4k on this -- http://amzn.to/2sTDnRZ\\nwith this lens -- http://amzn.to/2rUJOmD\\nbig drone - http://tinyurl.com/h4ft3oy\\nOTHER GEAR ---  http://amzn.to/2o3GLX5\\nSony CAMERA http://amzn.to/2nOBmnv\\nOLD CAMERA; http://amzn.to/2o2cQBT\\nMAIN LENS; http://amzn.to/2od5gBJ\\nBIG SONY CAMERA; http://amzn.to/2nrdJRO\\nBIG Canon CAMERA; http://tinyurl.com/jn4q4vz\\nBENDY TRIPOD THING; http://tinyurl.com/gw3ylz2\\nYOU NEED THIS FOR THE BENDY TRIPOD; http://tinyurl.com/j8mzzua\\nWIDE LENS; http://tinyurl.com/jkfcm8t\\nMORE EXPENSIVE WIDE LENS; http://tinyurl.com/zrdgtou\\nSMALL CAMERA; http://tinyurl.com/hrrzhor\\nMICROPHONE; http://tinyurl.com/zefm4jy\\nOTHER MI

In [12]:
noheader.map(lambda x: x.split(",")).take(10)

[[u'2kyS6SvSYSE',
  u'17.14.11',
  u'"WE WANT TO TALK ABOUT OUR MARRIAGE"',
  u'"CaseyNeistat"',
  u'22',
  u'2017-11-13T17:13:01.000Z',
  u'"SHANtell martin"',
  u'748374',
  u'57527',
  u'2966',
  u'15954',
  u'https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg',
  u'False',
  u'False',
  u'False',
  u'"SHANTELL\'S CHANNEL - https://www.youtube.com/shantellmartin\\nCANDICE - https://www.lovebilly.com\\n\\nfilmed this video in 4k on this -- http://amzn.to/2sTDnRZ\\nwith this lens -- http://amzn.to/2rUJOmD\\nbig drone - http://tinyurl.com/h4ft3oy\\nOTHER GEAR ---  http://amzn.to/2o3GLX5\\nSony CAMERA http://amzn.to/2nOBmnv\\nOLD CAMERA; http://amzn.to/2o2cQBT\\nMAIN LENS; http://amzn.to/2od5gBJ\\nBIG SONY CAMERA; http://amzn.to/2nrdJRO\\nBIG Canon CAMERA; http://tinyurl.com/jn4q4vz\\nBENDY TRIPOD THING; http://tinyurl.com/gw3ylz2\\nYOU NEED THIS FOR THE BENDY TRIPOD; http://tinyurl.com/j8mzzua\\nWIDE LENS; http://tinyurl.com/jkfcm8t\\nMORE EXPENSIVE WIDE LENS; http://tinyurl.com/zrdgtou\\n

In [13]:
import csv 
from StringIO import StringIO
from collections import namedtuple

In [14]:
field = header.replace(" ","_").replace("/", "_").split(",")
print field

[u'video_id', u'trending_date', u'title', u'channel_title', u'category_id', u'publish_time', u'tags', u'views', u'likes', u'dislikes', u'comment_count', u'thumbnail_link', u'comments_disabled', u'ratings_disabled', u'video_error_or_removed', u'description']


In [15]:
USA = namedtuple('USA', field, verbose=True)

class USA(tuple):
    'USA(video_id, trending_date, title, channel_title, category_id, publish_time, tags, views, likes, dislikes, comment_count, thumbnail_link, comments_disabled, ratings_disabled, video_error_or_removed, description)'

    __slots__ = ()

    _fields = ('video_id', 'trending_date', 'title', 'channel_title', 'category_id', 'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description')

    def __new__(_cls, video_id, trending_date, title, channel_title, category_id, publish_time, tags, views, likes, dislikes, comment_count, thumbnail_link, comments_disabled, ratings_disabled, video_error_or_removed, description):
        'Create new instance of USA(video_id, trending_date, title, channel_title, category_id, publish_time, tags, views, likes, dislikes, comment_count, thumbnail_link, comments_disabled, ratings_disabled, video_error_or_removed, description)'
       

In [16]:
def parse(row):
    reader = csv.reader(StringIO(row))
    row=reader.next()
    return USA(*row)

In [17]:
usa = noheader.map(parse)

In [18]:
usa.first()

USA(video_id='2kyS6SvSYSE', trending_date='17.14.11', title='WE WANT TO TALK ABOUT OUR MARRIAGE', channel_title='CaseyNeistat', category_id='22', publish_time='2017-11-13T17:13:01.000Z', tags='SHANtell martin', views='748374', likes='57527', dislikes='2966', comment_count='15954', thumbnail_link='https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg', comments_disabled='False', ratings_disabled='False', video_error_or_removed='False', description="SHANTELL'S CHANNEL - https://www.youtube.com/shantellmartin\\nCANDICE - https://www.lovebilly.com\\n\\nfilmed this video in 4k on this -- http://amzn.to/2sTDnRZ\\nwith this lens -- http://amzn.to/2rUJOmD\\nbig drone - http://tinyurl.com/h4ft3oy\\nOTHER GEAR ---  http://amzn.to/2o3GLX5\\nSony CAMERA http://amzn.to/2nOBmnv\\nOLD CAMERA; http://amzn.to/2o2cQBT\\nMAIN LENS; http://amzn.to/2od5gBJ\\nBIG SONY CAMERA; http://amzn.to/2nrdJRO\\nBIG Canon CAMERA; http://tinyurl.com/jn4q4vz\\nBENDY TRIPOD THING; http://tinyurl.com/gw3ylz2\\nYOU NEED THIS FOR T

In [19]:
usa.first().category_id

'22'

In [ ]:
usa.map(lambda x:x.category_id).countByValue()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 230, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 225, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 857, in func
    initial = next(iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 1271, in countPartition
    for obj in iterator:
  File "/usr/local/spark/p